# 事前準備

ライブラリのインストール

In [ ]:
!pip install numexpr

# 港湾統計マスタの前処理

港湾統計2021のcsvを読み込む

In [15]:
import pandas as pd
df_matster = pd.read_csv('kouwan2021.csv', header=0, encoding="shift-jis",dtype=str)
df_matster

,様式,調査年,調査月,調査港,申告者,調査票,整理番号,施設,公専,船舶内外,...,車両台数入,最初港,最終港,ｴﾗｰｺｰﾄﾞ,入港日,航路名,種類,種別,長さ,個数
0,A,2021,1,201,310,1,1,2350,2,2,...,0,NaN,NaN,0,3,31,NaN,NaN,NaN,0
1,A,2021,1,202,320,1,1,1241,1,1,...,0,NaN,NaN,0,7,1,NaN,NaN,NaN,0
2,A,2021,1,202,321,1,1,2308,2,1,...,0,NaN,NaN,0,7,2,NaN,NaN,NaN,0
3,A,2021,1,202,322,1,1,2352,2,1,...,0,NaN,NaN,0,10,2,NaN,NaN,NaN,0
4,A,2021,1,202,324,1,1,1212,1,1,...,0,NaN,NaN,0,8,1,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44826,A,2021,12,14,497,6,35,2488,2,2,...,0,NaN,NaN,0,24,31,NaN,NaN,NaN,0
44827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


必要な列だけ抽出

In [16]:
# 抽出する列のリスト指定
columns = ['調査港','施設','公専','用途','船舶隻数','船舶総ﾄﾝ数','係留時間','出荷品名','出荷ﾄﾝ数','入荷品名','入荷ﾄﾝ数']

df =  df_matster[columns]
df

,調査港,施設,公専,用途,船舶隻数,船舶総ﾄﾝ数,係留時間,出荷品名,出荷ﾄﾝ数,入荷品名,入荷ﾄﾝ数
0,201,2350,2,T74,1,113502,23,NaN,0,322,187122
1,202,1241,1,T84,0,0,3,NaN,0,161,1600
2,202,2308,2,T97,0,0,13,NaN,0,481,713
3,202,2352,2,T71,0,0,28,222,469,NaN,0
4,202,1212,1,t85,1,749,4,NaN,0,281,1502
...,...,...,...,...,...,...,...,...,...,...,...
44826,14,2488,2,T85,1,3215,11,281,5000,NaN,0
44827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


調査港、施設（岸壁）

In [17]:
# 抽出する港区コードのリスト指定
port_id = ['201','202','203','204','205','402','403','404','405','406']

df = df[df['調査港'].isin(port_id)]
df

,調査港,施設,公専,用途,船舶隻数,船舶総ﾄﾝ数,係留時間,出荷品名,出荷ﾄﾝ数,入荷品名,入荷ﾄﾝ数
0,201,2350,2,T74,1,113502,23,NaN,0,322,187122
1,202,1241,1,T84,0,0,3,NaN,0,161,1600
2,202,2308,2,T97,0,0,13,NaN,0,481,713
3,202,2352,2,T71,0,0,28,222,469,NaN,0
4,202,1212,1,t85,1,749,4,NaN,0,281,1502
...,...,...,...,...,...,...,...,...,...,...,...
44814,402,2216,2,T71,1,3305,11,NaN,0,191,4130
44815,402,2215,2,T71,0,0,10,NaN,0,341,1503
44816,403,2218,2,T71,0,0,11,222,246,NaN,0
44817,402,2216,2,T71,1,749,7,NaN,0,211,2100


対象施設コード

In [18]:
# 抽出する港区コードのリスト指定
shisetsu_id = ['1285','1270','1280','1275','1279','1203','1207','1222','1213','1214','1215','1219','1208','1231', \
               '1232','1241','1242','1223','1212','1220','1221','1249','1281','1250','1289','1269','1264','1263', \
               '1262','1258','1259','1252','1251','1254','1255','1256','1153','1152','1154','1197','1178','1179', \
               '1159','1160','1170','1171','1199','1181','1182','1195','1155','1156','1157','1158','1198']

df = df[df['施設'].isin(shisetsu_id)]
df

,調査港,施設,公専,用途,船舶隻数,船舶総ﾄﾝ数,係留時間,出荷品名,出荷ﾄﾝ数,入荷品名,入荷ﾄﾝ数
1,202,1241,1,T84,0,0,3,NaN,0,161,1600
4,202,1212,1,t85,1,749,4,NaN,0,281,1502
5,202,1221,1,T71,1,9910,64,NaN,0,121,10700
6,202,1231,1,T71,1,600,3,NaN,0,222,398
8,202,1215,1,T85,1,748,3,NaN,0,281,1002
...,...,...,...,...,...,...,...,...,...,...,...
44773,202,1249,1,T69,0,0,2,NaN,0,NaN,0
44774,202,1249,1,T69,0,0,0,NaN,0,NaN,0
44787,204,1256,1,T71,1,749,3,222,381,NaN,0
44788,204,1254,1,T71,0,0,4,NaN,0,481,666


# 取扱貨物量